# VegaChatGuide

## Pre-requisitos

In [ ]:
!pip install langchain
!pip install llama-index
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pyocr
!wget https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa.traineddata
!sudo mv "/content/spa.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"
!pip install onnxruntime
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.5/795.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: dataclasses-json
    Found existing installation: dataclasses-json 0.6.1
    Uninstalling dataclasses-json-0.6.1:
      Successfully uninstalled dataclasses-json-0.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages t

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPEN AI KEY"

## Entrenar con GPT-3.5-Turbo

In [ ]:
from llama_index import (
    LLMPredictor,
    GPTVectorStoreIndex,
    PromptHelper,
    ServiceContext,
    SimpleDirectoryReader
    )
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm = ChatOpenAI(
    temperature = 0.3,
    model_name = "gpt-3.5-turbo")
)

context_window = 4096
num_output = 256
chunk_overlap_ratio = 0.1

prompt_helper = PromptHelper(context_window, num_output, chunk_overlap_ratio)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

documents = SimpleDirectoryReader('./data').load_data()

index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist()

## Cargar modelo entrenado

In [ ]:
from llama_index import (
    LLMPredictor,
    ServiceContext,
    load_index_from_storage,
    StorageContext
)
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
storage_context = StorageContext.from_defaults(persist_dir='./directory')

index = load_index_from_storage(storage_context, service_context=service_context)

from llama_index.prompts import PromptTemplate

QA_PROMPT_TMPL = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question. If the answer is not in the context, inform "
    "the user that you can't answer the question - DO NOT MAKE UP AN ANSWER.\n"
    "In addition to returning the answer, also return a relevance score as to "
    "how relevant the answer is to the question. "
    "Question: {query_str}\n"
    "Answer (including relevance score): "
)
QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Telegram
- https://api.telegram.org/
- https://www.customvision.ai/projects/27b83612-fe12-4e88-acde-042e949b2e21#/performance
- https://platform.openai.com/account/billing/limits

In [ ]:
import telebot
import onnxruntime
import requests
import pytesseract
from PIL import Image
import os
import re
import cv2
import numpy as np
import openai
import json
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex
import time

# Guarda el tiempo actual cuando el bot se inicia
START_TIME = time.time()
# Constantes
BOT_TOKEN = "TELEGRAM BOT TOKEN"
MODEL_PATH = './object_detection_model.onnx'
STATUS_BAR_HEIGHT = 55
CONFIDENCE_THRESHOLD = 0.3

# Iniciar
bot = telebot.TeleBot(BOT_TOKEN)
#query_engine = index.as_query_engine()
query_engine = index.as_query_engine(text_qa_template=QA_PROMPT)

last_processed_update_id = None


bot.set_webhook()
session = onnxruntime.InferenceSession(MODEL_PATH, providers=['AzureExecutionProvider', 'CPUExecutionProvider'])

# Diccionario tipos de imagenes: cada tipo tiene una lista de palabras claves que el OCR busca para reconocer a que categoria pertenece una imagen
IMAGE_TYPES = {
    'Para solucionar el problema de GPS inmovil, asegúrate de tener la batería cargada a más del 15% y habilitar el GPS en la aplicación VMaps. Para hacer esto: \n\n1. Ve a la sección de ajustes de tu dispositivo. \n2. Busca y selecciona VMaps \n3. Ingresa a la sección de permisos y marca la opción "Permitir siempre". \n4. Reinicia el dispositivo. \n\nSi el problema persiste, te recomendamos contactar con un administrador para obtener ayuda adicional.': ['inmóvil', 'inmovil'],
    'Selecciona el botón verde "Registrar Aplicación". Después verás el mensaje: "Usted tiene que esperar a que el administrador valide y autorice la solicitud de registro del equipo". \n\nAdjunta una captura de pantalla y envíala a tu grupo de Whatsapp, un administrador se encargará del registro.': ['no se encuentra registrado en nuestros servidores', 'Registrar Aplicacion', 'Registrar Aplicación'],
    'Debes contactar con un administrador por Whatsapp para que apruebe tu registro o verifique si alguien está usando tu cuenta.': ['cuenta está siendo usado en otro dispositivo', 'existe una solicitud pendiente de aprobación', 'existe una solicitud pendiente de aprobacion'],
    'Debes: \n\n1. Ingresar a los ajustes de tu dispositivo. \n2. Actualizar la fecha y hora \n3. Reiniciar el dispositivo. \n\nUna vez que hayas corregido la fecha del sistema, podrás ingresar nuevamente a la aplicación VMaps.': ['fecha del sistema fue cambiado'],
    'Para iniciar sesión en VMaps: \n\n1. Asegurate de tener instalada la última versión del aplicativo V-Maps. \n2. Contacta a un administrador para que te registre en el sistema. \n3. Una vez registrado, abre el aplicativo e ingresa el nombre de tu empresa "VEGA", "valvoline" o "vegaallnes" según tu caso. Después tu usuario y contraseña (Asegurate de no dejar espacios en blanco). \n\nSi tienes problemas con tus credenciales, pregunta en tu grupo respectivo de Whatsapp.': ['Ingresa a tu cuenta', 'Usuario es requerido', 'Contraseña es requerida'],
    'Debes ir a tu grupo de Whatsapp y pedirle a un administrador que actualice tus datos en la plataforma V-Maps.': ['Usuario no encontrado'],
    'Si no existen clientes para la ruta seleccionada, es posible que demoren en cargar. Te recomendaría reiniciar VMaps y verificar si los clientes aparecen. \n\nSi el problema persiste, es necesario contactar con el área de planeamiento para obtener más información y resolver el inconveniente.': ['No existe clientes para la ruta'],
    'La opción "desarrollador" es una configuración que utilizan los desarrolladores para trabajar con el sistema de VMaps. \n\nTu como usuario, debes desactivar esta opción siguiendo los siguientes pasos: \n\n1. Ingresa a la configuración de la aplicación VMaps, \n2. Escribe "DESARROLLADOR" en la barra de búsqueda, \n3. Desactiva la opción. Si no desactivas esta opción, no podrás acceder a VMaps.': ['opción desarrollador', 'opcion desarrollador'],
    'Debe esperar a que el administrador valide y autorice su solicitud de registro en el equipo.': ['esperar a que el administrador valide y autorice la solicitud de registro del equipo', 'autorice la solicitud de registro del equipo'],
    'Para habilitar y verificar los permisos de GPS en VMaps, sigue estos pasos:\n\n1. Ve a la sección de "Ajustes" en tu dispositivo móvil. \n2. Busca y selecciona "Aplicaciones". \n3. Encuentra y selecciona "VMaps" en la lista de aplicaciones. \n4. Dentro de la sección de "Permisos", selecciona "Ubicación". \n5. Asegúrate de marcar la opción "Permitir siempre". \n6. Reinicia la aplicación VMaps. \n\nSi después de seguir estos pasos el problema persiste, te recomendamos contactar con un administrador para obtener ayuda adicional.': ['permisos para poder usar el GPS'],
    'Debes:\n\n1. Acceder al almacenamiento del equipo móvil \n2. Ir a Ajustes \n3. Almacenamiento. \n4. Seleccionar Aplicaciones y buscar V-Maps. \n5. Una vez allí, debe seleccionar "Borrar datos" \n6. Instalar la última versión del aplicativo V-Maps.': ['versión antigua', 'version antigua'],
    'Debes contactar con el Administrador por Whatsapp para que actualice tu contraseña, ya que el sistema bloquea la cuenta cuando se ingresan incorrectamente varias veces.': ['unusual activity'],
    'Debes revisar si cuentas con saldo disponible de internet en el celular. Luego de verificar, reinicia el celular y procede a ingresar nuevamente con las credenciales. \n\nSi tu problema no se soluciona, puede que sea por problemas del lado del servidor de VMaps, espera una respuesta de los administradores.': ['Unable to resolve host'],
    'Debes seguir los siguientes pasos para solucionar el problema "Usuario no fue encontrado en la tabla de roles":\n\n1. Verificar que tienes la última versión de VMaps. \n2. Acceder al almacenamiento del equipo móvil y dirigirte a la ruta /Ajustes/Almacenamiento/Aplicaciones/VMaps. \n3. Seleccionar la opción "Borrar datos". \n4. Ingresar nuevamente a VMaps y registrarte de nuevo utilizando el botón "Registrar Aplicación". \n5. Notificar a un administrador a través de Whatsapp para que apruebe tu registro.': ['no fue encontrado en la tabla de roles'],
    'Debes revisar si tienes saldo disponible de internet en el celular, luego de verificar, reiniciar el celular y proceder a ingresar luego con las credenciales. \n\nSi tu problema no se soluciona, puede que sea por problemas del lado del servidor de VMaps, espera una respuesta de los administradores.': ['conectarse con los servidores'],
}

# Preprocesamos la imagen con el fin de mejorar la captura del texto del OCR
def increase_contrast(image):
  image = image.resize((720, 1600))
  image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
  image_gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)

  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
  contrast_image = clahe.apply(image_gray)
  _, binarized_image = cv2.threshold(contrast_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  return Image.fromarray(binarized_image)

# Limpiar el texto para evitar caracteres raros como simbolos dolar, porcentaje, etc
def clean_text(text):
  cleaned_text = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ\s]', '', text)
  cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
  lines = cleaned_text.split('\n')
  lines = [line for line in lines if line.strip()]
  cleaned_text = '\n'.join(lines)

  return cleaned_text

def detect_image_types(text):
  detected_types = set()
  for image_type, keywords  in IMAGE_TYPES.items():
    for keyword  in keywords :
      if keyword  in text:
        detected_types.add(image_type)

  return list(detected_types)

def extract_text_from_image(image_path):
  image = Image.open(image_path)
  image = increase_contrast(image)

  text = pytesseract.image_to_string(image, lang="spa", config='--psm 6')
  text = clean_text(text)
  detected_image_types = detect_image_types(text)

  return detected_image_types, text

# Función para recortar la imagen y obtener solo la barra de estado
def crop_status_bar(image_path):
  image = Image.open(image_path)
  ancho_deseado = 720
  factor_escalado = ancho_deseado / image.width
  nuevo_alto = int(image.height * factor_escalado)
  image = image.resize((ancho_deseado, nuevo_alto))
  cropped_image = image.crop((0, 0, ancho_deseado, STATUS_BAR_HEIGHT))

  return cropped_image

# Obtener la lista de categorias de imagenes detectados
def process_image(image_path, mensaje):
  detected_objects_list, text = extract_text_from_image(image_path)

  imagen_recortada = crop_status_bar(image_path)
  imagen_recortada = imagen_recortada.resize((320, 320))
  imagen_recortada = np.array(imagen_recortada).astype('float32')
  imagen_recortada = np.transpose(imagen_recortada, (2, 0, 1))
  imagen_recortada = np.expand_dims(imagen_recortada, axis=0)

  input_name = session.get_inputs()[0].name
  predictions = session.run(None, {input_name: imagen_recortada})

  etiquetas_predichas = predictions[1][0]
  confianzas = predictions[2][0]

  etiquetas = ['Actualizacion sistema pendiente. Para actualizar el sistema de tu dispositivo, debes seguir los siguientes pasos: \n\n1. Ingresa a la configuración de tu dispositivo. \n2. Busca la opción de "Actualización de software" o "Actualización del sistema". \n3. Si hay una actualización pendiente, selecciona la opción para descargar e instalar la actualización. \n4. Asegúrate de tener una conexión estable a Internet durante el proceso de actualización. \n5. Una vez que la actualización se haya completado, reinicia tu dispositivo si es necesario. \n\nRecuerda que es importante mantener tu sistema operativo actualizado para garantizar un rendimiento óptimo y la corrección de posibles errores o vulnerabilidades de seguridad.',
               'Debes desactivar el modo ahorro de batería de tu dispositivo, de lo contrario tu GPS no funcionará a su máxima capacidad',
               'Procura desactivar bluetooth debido a que este consume recursos como la bateria y afecta el rendimiento del GPS',
               'Procura Debes desactivar las notificaciones aplicaciones secundarias como Youtube, Tik tok, Facebook para ahorrar bateria y permitir al GPS funcionar a máxima capacidad.']

  for i, etiqueta_id in enumerate(etiquetas_predichas):
    confianza = confianzas[i]
    if confianza >= CONFIDENCE_THRESHOLD:
      etiqueta = etiquetas[etiqueta_id]
      detected_objects_list.append(etiqueta)

  detected_objects_list = list(set(detected_objects_list))

  return detected_objects_list if detected_objects_list else None


@bot.message_handler(content_types=['text', 'photo'])
def handle_messages(mensaje):
    # Obtén el tiempo de envío del mensaje
    message_time = mensaje.date

    # Compara el tiempo de envío con el tiempo de inicio del bot
    if message_time < START_TIME:
        return  # Ignora el mensaje si es más antiguo que el tiempo de inicio

    if mensaje.content_type == 'text':
        handle_message(mensaje)

    elif mensaje.content_type == 'photo':
        handle_photo(mensaje)

@bot.message_handler(content_types=['text'])
def handle_message(mensaje):

  texto = mensaje.text
  response = query_engine.query(texto + ". Reply in Español.")
  bot.send_message(mensaje.chat.id, response.response)

@bot.message_handler(content_types=['photo'])
def handle_photo(mensaje):
  chat_id = mensaje.chat.id

  file_info = bot.get_file(mensaje.photo[-1].file_id)
  file_url = f"https://api.telegram.org/file/bot{BOT_TOKEN}/{file_info.file_path}"

  response = requests.get(file_url)
  with open("temp_image.jpg", "wb") as image_file:
    image_file.write(response.content)

  detected_objects_list = process_image("temp_image.jpg", mensaje)

  if detected_objects_list:
    for etiqueta in detected_objects_list:
      #response = query_engine.query(etiqueta + ". Reply in Español.")
      bot.send_message(chat_id, etiqueta)

  if mensaje.caption:
    texto = mensaje.caption
    response = query_engine.query(texto + ". Reply in Español.")
    bot.send_message(mensaje.chat.id, response.response)

  elif not detected_objects_list and not mensaje.caption:
    response_text = "Lo siento, no pude comprender el problema en la screenshot que enviaste. Por favor, recuerda que solo atiendo casos sobre la aplicación VMaps, si es posible envía un mensaje explicando tu duda o problema para que pueda ayudarte."
    bot.send_message(chat_id, response_text)

# Inicia el bot
bot.polling()